In [13]:
import torch
import os
import pandas as pd
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS

In [6]:
dataset_path = '/home/pill/competition/dataset'
train_images = '/home/pill/competition/dataset/public_train/pill/images'
train_labels = '/home/pill/competition/dataset/public_train/pill/labels'

In [10]:
def list_all_files(dir, ext = '.jpg'):
    files = []
    for r, d, f in os.walk(dir):
        for file in f:
            if file.endswith(ext):
                files.append(os.path.join(r, file))
    return files

In [16]:
images_dir = list_all_files(train_images)
images_dir

['/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_487_5.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_471_8.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_545_10.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_13_1.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_501_0.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_310_0.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_350_14.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_567_10.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_328_1.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_33_10.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_551_2.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_1058_4.jpg',
 '/home/pill/competition/dataset/public_train/pill/images/VAI

In [19]:
image = Image.open('/home/pill/competition/dataset/public_train/pill/images/VAIPE_P_487_5.jpg')
exifdata = image.getexif()
for tag_id in exifdata:
    # get the tag name, instead of human unreadable tag id
    tag = TAGS.get(tag_id, tag_id)
    data = exifdata.get(tag_id)
    # decode bytes 
    if isinstance(data, bytes):
        data = data.decode()
    print(f"{tag:25}: {data}")

ImageWidth               : 4000
ImageLength              : 3000
ImageDescription         :                                
Make                     : samsung                        
Model                    : SM-A225F                       
Orientation              : 6
YCbCrPositioning         : 2
XResolution              : 72.0
YResolution              : 72.0
                      544: 0
                      545: 0
                      546: 0
                      547: 0
                      548: 0
                      549:                                
ResolutionUnit           : 2
Software                 : A225FXXU2AUH1                  
DateTime                 : 2022:03:10 02:21:49
ExifOffset               : 438
